In [4]:
# config.py
data_path = "aurora_digits"
digit_names = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'O', 'Z']

filename_index_map = {
    '1': 0,
    '2': 1,
    '3': 2,
    '4': 3,
    '5': 4,
    '6': 5,
    '7': 6,
    '8': 7,
    '9': 8,
    'O': 9,
    'Z': 10,
}

isolated_model_path = 'isolated_models'
continuous_model_path = 'continuous_models'

In [3]:
try:
    os.mkdir(isolated_model_path)
except:
    print("Warnings:", isolated_model_path, "already exits?")
    
try:
    os.mkdir(continuous_model_path)
except:
    print("Warnings:", continuous_model_path, "already exits?")

In [5]:
# feature.py
import numpy as np
from scipy.io import wavfile
from scipy.fftpack import dct
import math


def segment(sig, sample_rate=16000, frame_len=0.025, frame_step=0.01):
    slen = sig.size
    frame_len1 = int(frame_len * sample_rate)
    frame_step1 = int(frame_step * sample_rate)
    num_frames = 1
    if slen > frame_len:
        num_frames = math.ceil(slen / frame_step1)

    final_len = int((num_frames - 1) * frame_step1 + frame_len1)

    pad_sig = np.concatenate([sig, np.zeros(final_len - slen)])
    frames = np.zeros((num_frames, frame_len1))
    for i in range(num_frames):
        frames[i, :] = pad_sig[i * frame_step1:i * frame_step1 + frame_len1]

    return frames


def zero_padding(frames, frame_len=None):
    width = frames.shape[1]
    height = frames.shape[0]

    # if frame_len is not specified, find the next power of 2
    if frame_len is None:
        frame_len = 1 << (width - 1).bit_length()

    pad_len = frame_len - width
    pad_len_left = pad_len // 2
    pad_len_right = pad_len - pad_len_left

    f = np.zeros((frames.shape[0], frame_len))
    for i in range(0, height):
        f[i, pad_len_left:frame_len - pad_len_right] = frames[i, :]
    return f


def mfcc_features(path_file, frame_size=0.025, frame_stride=0.01, low_freq=80, high_freq=None):
    sample_rate, signal = wavfile.read(path_file)
    pre_emphasis = 0.97
    emphasized_signal = np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])

    frames = segment(emphasized_signal, sample_rate, frame_size, frame_stride)
    frames = zero_padding(frames)

    # hamming window
    frames *= np.hamming(frames.shape[1])

    NFFT = 512
    mag_frames = np.absolute(np.fft.rfft(frames, NFFT))  # Magnitude of the FFT
    pow_frames = ((1.0 / NFFT) * (mag_frames ** 2))  # Power Spectrum

    nfilt = 40
    high_freq = high_freq or sample_rate / 2
    low_freq_mel = (2595 * np.log10(1 + low_freq / 700))  # Convert Hz to Mel
    high_freq_mel = (2595 * np.log10(1 + high_freq / 700))  # Convert Hz to Mel
    mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
    hz_points = (700 * (10 ** (mel_points / 2595) - 1))  # Convert Mel to Hz
    bin = np.floor((NFFT + 1) * hz_points / sample_rate)

    fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])  # left
        f_m = int(bin[m])  # center
        f_m_plus = int(bin[m + 1])  # right

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    filter_banks = np.dot(pow_frames, fbank.T)
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # Numerical Stability
    filter_banks = np.log10(filter_banks)

    num_ceps = 13
    mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1: (num_ceps + 1)]
    return filter_banks, mfcc


def standardize(data):
    data -= (np.mean(data, axis=0))
    data = data / np.std(data, axis=0)
    return data

In [6]:
# dtw.py
import numpy as np
from math import isinf


def dtw(x, y, dist_fun, transitions, variance=None, beam=np.inf):
    """
    :param x: an input
    :param y: a template
    :param dist_fun: function used to calculate distance between two nodes.
    :param transitions: transition matrix, where transitions[i,j] represents the cost for transiting from jth to ith
        state.
    :param variance: covariance matrix of the segments in the templates.
    :param beam: beam size of pruning.
    :return: costs: cost matrix.
            path: reversed path (from end to start). [[x_n,y_n],...,[x_2,y_2],[x_1,y_1]]
    """
    # initialize cost matrix
    col_count = len(x)
    row_count = len(y)
    assert col_count > 1 and row_count > 1
    costs = np.full((row_count, col_count), np.inf)
    path_matrix = np.full((row_count, col_count, 2), np.inf, dtype=np.int)

    j = 0
    while True:
        if j == col_count:
            break
        i = 0
        while True:
            if i == row_count:
                break
            if i == 0 and j == 0:
                if variance is None:
                    costs[0, 0] = dist_fun(x[j], y[i])
                else:
                    costs[0, 0] = dist_fun(x[j], y[i], variance[i])
                i += 1
                continue

            prev_costs = []
            from_pts = []
            for origin in range(transitions.shape[1]):
                # cost is set to -1 if it is excluded from beam search
                if costs[origin, j - 1] == -1:
                    # set it back to np.inf so that it is more convenient to be recognized by pathfinder
                    costs[origin, j - 1] = np.inf
                else:
                    prev_costs.append(transitions[i, origin] + costs[origin, j - 1])
                    from_pts.append([origin, j - 1])

            min_i = np.argmin(prev_costs)
            origin = from_pts[min_i]
            path_matrix[i, j] = origin
            if variance is None:
                curr_cost = prev_costs[min_i] + dist_fun(x[j], y[i])
            else:
                curr_cost = prev_costs[min_i] + dist_fun(x[j], y[i], variance[i])
            costs[i, j] = min(costs[i, j], curr_cost)
            i += 1
        if not isinf(beam):
            sort_idx = np.argsort(costs[:, j].flatten())
            # cost is set to -1 if it is excluded from beam search
            exclude_idx = sort_idx[beam:]
            for i in exclude_idx:
                if not isinf(costs[i, j]):
                    costs[i, j] = -1
        j += 1
    # find the path
    i = row_count - 1
    j = col_count - 1
    path = []
    while i != 0 or j != 0:
        i, j = path_matrix[i, j]
        path.append([i, j])
    return costs, np.array(path)


def dtw1(x, states, transitions, beam=np.inf):
    """
    :param x: an input
    :param states: a list of states (GMM object).
    :param transitions: transition matrix, where transitions[i,j] represents the cost for transiting from jth to ith
        state.
    :param beam: beam size of pruning.
    :return: costs: cost matrix.
            path: reversed path (from end to start). [[x_n,y_n],...,[x_2,y_2],[x_1,y_1]]
    """
    # initialize cost matrix
    col_count = len(x)
    row_count = len(states)
    costs = np.full((row_count, col_count), np.inf)
    path_matrix = np.full((row_count, col_count, 2), np.inf, dtype=np.int)

    j = 0
    while True:
        if j == col_count:
            break
        i = 0
        while True:
            if i == row_count:
                break
            if i == 0 and j == 0:
                costs[0, 0] = states[i].evaluate(x[j])
                i += 1
                continue

            prev_costs = []
            from_pts = []
            for origin in range(transitions.shape[1]):
                # cost is set to -1 if it is excluded from beam search
                if costs[origin, j - 1] == -1:
                    # set it back to np.inf so that it is more convenient to be recognized by pathfinder
                    costs[origin, j - 1] = np.inf
                else:
                    prev_costs.append(transitions[i, origin] + costs[origin, j - 1])
                    from_pts.append([origin, j - 1])

            min_i = np.argmin(prev_costs)
            origin = from_pts[min_i]
            path_matrix[i, j] = origin
            curr_cost = prev_costs[min_i] + states[i].evaluate(x[j])
            costs[i, j] = min(costs[i, j], curr_cost)
            i += 1
        if not isinf(beam):
            sort_idx = np.argsort(costs[:, j].flatten())
            # cost is set to -1 if it is excluded from beam search
            exclude_idx = sort_idx[beam:]
            for i in exclude_idx:
                if not isinf(costs[i, j]):
                    costs[i, j] = -1
        j += 1
    # find the path
    i = row_count - 1
    j = col_count - 1
    path = []
    while i != 0 or j != 0:
        i, j = path_matrix[i, j]
        path.append([i, j])
    return costs, np.array(path)


def sentence_viterbi(x, model_graph):
    # TODO: allow arbitrary jumps between gmm states depending on the hmm model
    """
    :param x: input features.
    :param model_graph: TODO: add docstring
    :return: costs: cost matrix
            matched_sentence: a sequence of index of models that best matches the input.

    """
    nodes = model_graph.nodes

    # initialize cost matrix
    n_cols = len(x)
    n_rows = len(nodes)
    costs = np.full((n_rows, n_cols), np.inf)
    costs[0, 0] = 0

    path_matrix = np.full((n_rows, n_cols, 2), np.inf, dtype=np.int)

    # the end of a sentence
    end_nodes = model_graph.get_ends()

    # fill cost matrix
    for c in range(n_cols):
        for r in range(0, n_rows):
            if r == 0 and c == 0:
                continue
            subcosts = []
            from_points = []

            if model_graph[r].model_index == 'NES':
                # the cost of non-emitting state is 0
                node_dist = 0
            else:
                node_dist = model_graph[r].val.evaluate(x[c])

            origins, transition_costs = model_graph.get_origins(model_graph[r])

            for o, tc in zip(origins, transition_costs):
                origin_idx = o.node_index
                if model_graph[r].model_index == 'NES':
                    subcosts.append(costs[origin_idx, c] + node_dist)
                    from_points.append([origin_idx, c])
                elif c > 0:
                    subcosts.append(tc + costs[origin_idx, c - 1] + node_dist)
                    from_points.append([origin_idx, c - 1])

            # if there is no node that can get to current position, simply skip it.
            if len(subcosts) == 0:
                continue
            # remember path and cost
            min_idx = np.argmin(subcosts)
            path_matrix[r, c] = from_points[min_idx]
            costs[r, c] = subcosts[min_idx]

    # find the sentence which has the min cost
    end_node_costs = [costs[n.node_index, -1] for n in end_nodes]
    min_idx = np.argmin(end_node_costs)
    best_end_idx = end_nodes[min_idx].node_index
    best_cost = costs[best_end_idx, -1]

    # find the matched string
    c = n_cols - 1
    r = best_end_idx
    matched_sentence = [nodes[r].model_index]
    while 1:
        if c < 1:
            break
        r, c = path_matrix[r, c]
        if r != 0:
            matched_sentence.append(nodes[r].model_index)

    return best_cost, matched_sentence[::-1]


In [7]:
# kmeans.py
import numpy as np


def calc_variance(data):
    return np.cov(data).diagonal()


def combine_templates(templates, n_temps, n_segments, seg_starts):
    """
    :param templates: a list of templates
    :param n_temps: the number of templates
    :param n_segments: the number of segments the final templates will have
    :param seg_starts: the start points of the segments in the original templates
    :return: the combined templates
    """
    res = np.zeros((n_segments, templates[0].shape[1]))
    vars = np.zeros((n_segments, templates[0].shape[1]))
    segments = segment_data(templates, n_temps, n_segments, seg_starts)
    for s in range(n_segments):
        seg = segments[s]
        res[s] = seg.mean(axis=0)
        vars[s] = calc_variance(seg.T)
    return res, vars


def segment_data(templates, n_temps, n_segments, seg_starts):
    """
    :param templates: a list of templates
    :param n_temps: the number of templates
    :param n_segments: the number of segments the final templates will have
    :param seg_starts: the start points of the segments in the original templates
    :return: the segmented data
    """
    segments = []
    for s in range(n_segments):
        seg = []
        for r in range(n_temps):
            if s == n_segments - 1:
                seg += templates[r][seg_starts[r, s]:].tolist()
            else:
                seg += templates[r][seg_starts[r, s]:seg_starts[r, s + 1]].tolist()
        segments.append(np.array(seg))
    return segments


def calc_transition_costs(templates, seg_lens, max_jump_dist=2):
    """
    Calculate and construct a dictionary used in dtw()
    :param templates: a list of templates
    :param seg_lens: list of length of segments of all templates
    :param max_jump_dist: maximum distance allowed for a transition to jump across, the default value 2 is optimal in
        most occasions.
    :return: the transition cost
    """
    n_segments = seg_lens.shape[1]
    n_temps = len(templates)
    empty_segs = (seg_lens == 0)
    res = np.full((n_segments, n_segments), np.inf)
    for i in range(n_segments):
        jump_dist = 1
        # number of samples which transition to other segments
        if i == n_segments - 1:
            n_jump = 0
        else:
            n_jump = n_temps
        s = i + 1
        # calculate the distance of the jump
        while s < n_segments - 1:
            # if the next segments is not empty, the jump stops here; otherwise increment jump_dist
            if np.sum(empty_segs[:, s + 1]) == 0:
                break
            jump_dist += 1
            if jump_dist > max_jump_dist:
                break
            s += 1
        # number of samples in this segment
        n_all = 0
        for t in range(len(templates)):
            n_all += seg_lens[t, i]
        # number of samples which transition to itself
        n_stay = n_all - n_jump
        p_stay = n_stay / n_all
        p_jump = n_jump / n_all
        if n_jump:
            res[i + jump_dist, i] = -np.log(p_jump)
        res[i, i] = -np.log(p_stay)
    return res


def get_segments_from_path(path, n_segments):
    # count all segments occurrence
    unique, counts = np.unique(path[:, 0], return_counts=True)
    seg_counts = dict(zip([i for i in range(n_segments)], [0 for _ in range(n_segments)]))
    seg_counts.update(dict(zip(unique, counts)))
    counts = np.array(list(seg_counts.values()))
    unique = np.array(list(seg_counts.keys()))
    sort_idx = np.argsort(unique)
    counts = counts[sort_idx]
    seg_starts = np.add.accumulate(counts)[:-1]
    return seg_starts


def skmeans(templates, n_segments, dist_fun=lambda *args: np.linalg.norm(args[0] - args[1]),
            return_segmented_data=False, max_iteration=1000):
    """
    :param templates: templates as a list
    :param n_segments: number of segments in the final template
    :param dist_fun: function to calculate distance of two nodes in the templates
    :param return_segmented_data: if true, return segmented data, and vice versa.
    :param max_iteration: max iteration if not converged
    :return: res: a combined template;
            vars: the variance of all segments;
            transition_costs: cost matrix of transitions;
            if return_segmented_data is set to true, segmented_data is returned, it is a list of all segments.
    """
    assert max_iteration > 0
    n_temps = len(templates)
    seg_lens = np.zeros((n_temps, n_segments + 1), dtype=np.int)
    for r in range(n_temps):
        temp_len = len(templates[r])
        seg_len = temp_len // n_segments
        seg_lens[r, 1:] = seg_len
    seg_starts = np.add.accumulate(seg_lens, axis=1)[:, :-1]
    seg_lens = seg_lens[:, 1:]

    res, vars = combine_templates(templates, n_temps, n_segments, seg_starts)
    for _ in range(max_iteration):
        # do dtw and align templates
        seg_starts = np.zeros((n_temps, n_segments), dtype=np.int)
        transition_costs = calc_transition_costs(templates, seg_lens)
        for r in range(n_temps):
            # if template contains too few mfcc vectors, we cannot find a path in dtw
            if templates[r].shape[0] < 5:
                raise NameError('template is too small, cannot do dtw on it')

            _, path = dtw(templates[r], res, dist_fun, transition_costs)
            seg_starts[r, 1:] = get_segments_from_path(path, n_segments)
        new_res, vars = combine_templates(templates, n_temps, n_segments, seg_starts)
        if np.allclose(res, new_res):
            break
        res = new_res
    if return_segmented_data:
        segmented_data = segment_data(templates, n_temps, n_segments, seg_starts)
        return res, vars, transition_costs, segmented_data
    else:
        return res, vars, transition_costs


def cluster_centroids(data, clusters, k):
    """Return centroids of clusters in data.
    """
    result = np.empty(shape=(k,) + data.shape[1:])
    for i in range(k):
        np.mean(data[clusters == i, :], axis=0, out=result[i])
    return result


def kmeans(data, k, centroids, dist_fun=lambda *args: np.linalg.norm(args[0] - args[1]), max_iteration=1000):
    """Modified k-means algorithm to fit the scenario
    """
    assert k == centroids.shape[0]
    clusters = np.random.randint(0, k, data.shape[0])
    # find the covariance matrix of the new clusters
    cov = []
    for c in range(k):
        segments = data[clusters == c]
        cov.append(calc_variance(segments.T))
    cov = np.array(cov)
    # calculate distance from centroids
    for _ in range(max(max_iteration, 1)):
        dists = np.zeros((data.shape[0], k))
        for i in range(data.shape[0]):
            for c in range(k):
                dists[i, c] = dist_fun(centroids[c, :], data[i, :], cov[0])
        dists = np.array(dists)
        # Index of the closest centroid to each data point.
        clusters = np.argmin(dists, axis=1)
        new_centroids = cluster_centroids(data, clusters, k)

        # check convergence
        if np.array_equal(new_centroids, centroids):
            break
        centroids = new_centroids
    return clusters, centroids, cov


def align_gmm_states(templates, gmm_states, transition_costs, n_segments):
    n_temps = len(templates)

    # do dtw and align templates
    seg_starts = np.zeros((n_temps, n_segments), dtype=np.int)
    for r in range(n_temps):
        t = templates[r]
        _, path = dtw1(t, gmm_states, transition_costs)
        seg_starts[r, 1:] = get_segments_from_path(path, n_segments)
    return segment_data(templates, n_temps, n_segments, seg_starts)

In [8]:
# hmm.py
import numpy as np
import copy
import scipy


class MultivariateNormal:
    def __init__(self, mean, cov):
        self.mean = mean
        self._cov = cov
        if len(self.cov.shape) == 1:
            cov = np.diag(self.cov)
        else:
            cov = self.cov
        self.inv_cov = np.linalg.inv(cov)

    @property
    def cov(self):
        return self._cov

    @cov.setter
    def cov(self, val):
        self._cov = val
        if len(self.cov.shape) == 1:
            cov = np.diag(self.cov)
        else:
            cov = self.cov
        self.inv_cov = np.linalg.inv(cov)

    @cov.deleter
    def cov(self):
        del self.cov

    def pdf(self, x):
        size = x.shape[0]
        if size == self.mean.shape[0]:
            det = np.prod(self.cov)
            norm_const = 1.0 / (np.power((2 * np.pi), float(size) / 2) * np.sqrt(det))
            x_mu = x - self.mean
            result = np.exp(-0.5 * (x_mu.dot(self.inv_cov).dot(x_mu.T)))
            return norm_const * result
        else:
            raise NameError("The dimensions of the input don't match")


def dist(v1, v2, variance):
    D = len(variance)
    m = (v1 - v2)
    return 0.5 * np.log((2 * np.pi) ** D * np.prod(variance)) + 0.5 * np.sum(m / variance * m)


class GMM:
    def __init__(self, mu, sigma, n_gaussians):
        self.n_gaussians = n_gaussians
        self.w = np.full(n_gaussians, 1 / n_gaussians)
        self.dists = [MultivariateNormal(mean=mu, cov=sigma) for _ in range(n_gaussians)]
        self.mu_old = np.tile(mu, (n_gaussians, 1))
        self.sigma_old = np.tile(sigma, (n_gaussians, 1))
        self.w_old = np.full(n_gaussians, 1 / n_gaussians)

    def evaluate(self, x, return_neg_log_likelihood=True):
        res = [self.dists[i].pdf(x) * self.w[i] for i in range(self.n_gaussians)]
        res = np.array(res)
        if return_neg_log_likelihood:
            return -np.log(res.sum())
        else:
            return res

    def em(self, data, n_gaussians, max_iteration=10000):
        for iter in range(max_iteration):
            p = np.zeros((data.shape[0], n_gaussians))
            mu = np.zeros((data.shape[1], n_gaussians))
            sigma = np.zeros((data.shape[1], n_gaussians))
            for i in range(data.shape[0]):
                p[i, :] = self.evaluate(data[i, :], return_neg_log_likelihood=False)[:n_gaussians]

            p_sum = np.sum(p, axis=1).reshape((p.shape[0], 1))
            # avoid divide by 0
            p_sum[p_sum == 0] = 10 ** (-5)
            p /= p_sum
            p_sum = np.sum(p, axis=0)
            # avoid divide by 0
            p_sum[p_sum == 0] = 10 ** (-5)
            for c in range(n_gaussians):
                mu[:, c] = np.sum(data * p[:, [c]], axis=0)
                mu[:, c] /= p_sum[c]
                # sigma
                sub2 = (data - mu[:, c]) ** 2
                sigma[:, c] = np.sum(sub2 * p[:, [c]], axis=0)
                sigma[:, c] /= p_sum[c]

            mu = mu.T
            sigma = sigma.T
            # update w and mu
            w = p.mean(axis=0).T
            self.update_models(mu, sigma, w)
            if np.allclose(mu, self.mu_old[:n_gaussians, :]) and \
                    np.allclose(sigma, self.sigma_old[:n_gaussians, :]) and \
                    np.allclose(w, self.w_old[:n_gaussians]):
                print("EM converged at iteration:", iter)
                break
            else:
                print("EM iteration:", str(iter), end="\r")
                self.mu_old[:n_gaussians, :] = mu
                self.sigma_old[:n_gaussians, :] = sigma
                self.w_old[:n_gaussians] = w

    def update_models(self, mus, sigmas, weights):
        self.w[:mus.shape[0]] = weights
        for m in range(mus.shape[0]):
            self.dists[m].mean = mus[m, :]
            self.dists[m].cov = sigmas[m, :]

    def __eq__(self, other):
        res = True
        res = res and self.n_gaussians == other.n_gaussians and np.allclose(self.w, other.w)
        for i in range(self.n_gaussians):
            res = res and np.allclose(self.dists[i].mean, other.dists[i].mean) and np.allclose(self.dists[i].cov,
                                                                                               other.dists[i].cov)
        return res


class HMM:
    """
    Attributes
    ----------
    mu: an array of means of all segments

    sigma: an array of variance of all segments
    transitions: transition matrix

    segments: a list of mfcc feature vectors of each segments
    """

    def __init__(self, n_segments):
        self.n_segments = n_segments
        self.mu = None
        self.sigma = None
        self.transitions = None
        self.segments = []
        self.gmm_states = None
        self.use_gmm = True
        self.use_em = True

    def __eq__(self, other):
        if self.use_gmm:
            if not other.use_gmm:
                return False
            if self.n_segments != other.n_segments:
                return False
            for i in range(self.n_segments):
                if self.gmm_states[i] != other.gmm_states[i]:
                    return False
            return True
        else:
            return np.allclose(self.mu, other.mu) and np.allclose(self.sigma, other.sigma)

    def reset(self):
        self.mu = None
        self.sigma = None
        self.transitions = None
        self.segments = []
        self.gmm_states = None

    def __getitem__(self, item):
        if type(item) is int or type(item) is slice:
            return self.gmm_states[item]
        else:
            raise TypeError('The type of index is not supported')

    def fit(self, ys, n_gaussians, use_gmm=True, use_em=True):
        """Fit the HMM model with a list of training data.
        :param use_gmm: TODO: add docstring
        :param use_em: if true, use both k-means and Expectation-Maximization algorithm to fit GMM, otherwise only
            use k-means to do it.
        :param ys: a list of mfcc templates. It cannot be a numpy array since variable length rows in a matrix are
            not supported.
        :param n_gaussians: the number of gaussian distributions. It should be some power of 2, if not it will be
            converted to previous power of 2.
        :return: the trained HMM model.
        """
        self.use_em = use_em
        self.use_gmm = use_gmm
        if use_gmm:
            self.fit_GMM(ys, n_gaussians)
        else:
            self.mu, self.sigma, self.transitions, self.segments = skmeans(ys, self.n_segments,
                                                                           return_segmented_data=True)
        return self

    def _init_gmm(self, n_gaussians):
        self.gmm_states = [GMM(self.mu[i, :], self.sigma[i, :], n_gaussians) for i in range(self.n_segments)]

    def fit_GMM(self, ys, n_gaussians):
        """fit all GMM states in the HMM.
        :param n_gaussians: the number of gaussians.
        """
        print('Doing segmental k-means')
        self.mu, self.sigma, self.transitions, self.segments = skmeans(ys, self.n_segments,
                                                                       return_segmented_data=True)
        self._init_gmm(n_gaussians)

        # train each GMM
        for i in range(len(self.segments)):
            self._fit_GMM(self.segments[i], n_gaussians, i)

        # update segments
        self.segments = align_gmm_states(ys, self.gmm_states, self.transitions, self.n_segments)

    def _fit_GMM(self, data, n_gaussians, seg_i):
        """fit a single GMM state. Only for internal use.
        :param data: the data only for this state.
        :param n_gaussians: the number of gaussians.
        :param seg_i: the index of this segment/state.
        :return: a GMM object.
        """
        n_splits = int(np.log(n_gaussians))
        assert n_splits > 0

        centroids = np.array([self.mu[seg_i, :]])
        weights = np.full(n_gaussians, 1 / data.shape[0])
        for i in range(n_splits):
            lcentroids = centroids * 0.9
            hcentroids = centroids * 1.1
            centroids = np.concatenate([lcentroids, hcentroids], axis=0)
            clusters, centroids, variance = kmeans(data, 2 ** (i + 1), centroids, dist_fun=dist)

            # calculate and update weights of distributions
            cs, c_counts = np.unique(clusters, return_counts=True)
            for c in cs:
                weights[c] = c_counts[c] / data.shape[0]

            # update model parameters
            self.gmm_states[seg_i].update_models(centroids, variance, weights[:2 ** (i + 1)])
            # Expectation-maximization
            if self.use_em:
                self.gmm_states[seg_i].em(data, 2 ** (i + 1))

    def evaluate(self, x):
        """
        :param x: input data.
        :return: cost/score/probability.
        """
        if self.use_gmm:
            costs, _ = dtw1(x, self.gmm_states, self.transitions)
        else:
            costs, _ = dtw(x, self.mu, dist, self.transitions, self.sigma)
        return costs[-1, -1]

In [9]:
# model_graph.py
import uuid


class GraphNode:
    def __init__(self, val, model_index=None, node_index=None):
        self.val = val
        self.model_index = model_index
        self.node_index = node_index
        # help distinguish different nodes
        self.id = uuid.uuid4().int

    def __eq__(self, other):
        return self.id == other.id


# TODO: make apis of LayeredHMMGraph ContinuousGraph the same
class Graph:
    def __init__(self, nodes):
        self.nodes = nodes
        self.edges = []

    def add_edge(self, from_, to, val=0):
        if from_ not in self.nodes:
            self.nodes.append(from_)
        if to not in self.nodes:
            self.nodes.append(to)
        self.edges.append((from_, to, val))

    def get_dests(self, origin):
        res = []
        values = []
        for o, d, val in self.edges:
            if o.id == origin.id:
                res.append(d)
                values.append(val)
        return res, values

    def get_origins(self, dest):
        res = []
        values = []
        for o, d, val in self.edges:
            if d.id == dest.id:
                res.append(o)
                values.append(val)
        return res, values

    def get_ends(self):
        raise NotImplemented()

    def __getitem__(self, item):
        return self.nodes[item]


class LayeredHMMGraph(Graph):
    def __init__(self, nodes):
        super().__init__(nodes)
        self.curr_layer = []
        self.ends = []

    def add_non_emitting_state(self, end=False):
        nes = GraphNode(None, model_index='NES', node_index=len(self.nodes))
        self.nodes.append(nes)  # NES stands for non-emitting state
        if len(self.curr_layer) > 0:
            for m in self.curr_layer:
                self.add_edge(m, nes)
        self.curr_layer = [nes]
        if end:
            self.ends.append(nes)
        return nes

    def add_layer_from_models(self, models):
        # assert that the previous layer is a non-emitting state
        assert len(self.curr_layer) == 1 and self.curr_layer[0].model_index == 'NES'
        new_layer = []
        for i in range(len(models)):
            m = models[i]
            # get gmm states of all models
            gmm_nodes = [GraphNode(gs, model_index=i) for gs in m.gmm_states]
            n_gaussians = len(gmm_nodes)
            # connect previous layer to the current one
            self.add_edge(self.curr_layer[0], gmm_nodes[0], 0)

            # build graph for all gmm states in each hmm model
            for j in range(0, n_gaussians):
                # self loop
                self.add_edge(gmm_nodes[j], gmm_nodes[j], val=m.transitions[j, j])
                if j < n_gaussians - 1:
                    # connect to the next
                    self.add_edge(gmm_nodes[j], gmm_nodes[j + 1], val=m.transitions[j + 1, j])
                else:
                    new_layer.append(gmm_nodes[j])
        # update self.curr_layer to the last gmm_states in models
        self.curr_layer = new_layer
        self.update_node_indices()
        return new_layer

    # TODO: don't update all nodes when unnecessary
    def update_node_indices(self):
        n_nodes = len(self.nodes)
        for i in range(n_nodes):
            if self.nodes[i].node_index is not None:
                assert self.nodes[i].node_index == i
            self.nodes[i].node_index = i

    def get_ends(self):
        return self.ends


class ContinuousGraph(Graph):
    def __init__(self, nodes):
        super().__init__(nodes)
        self.current = None

    def add_non_emitting_state(self):
        nes = GraphNode(None, 'NES')
        self.nodes.append(nes)  # NES stands for non-emitting state
        if self.current is not None:
            self.add_edge(self.current, nes)
        self.current = nes
        # set node_index
        nes.node_index = len(self.nodes) - 1
        return nes

    def add_model(self, model, model_index):
        # get gmm states of all models
        gmm_nodes = [GraphNode(model.gmm_states[i], model_index=model_index) for i in
                     range(len(model.gmm_states))]
        n_segments = len(gmm_nodes)
        # connect previous layer to the current one
        self.add_edge(self.current, gmm_nodes[0], val=0)

        # build graph for all gmm states in each hmm model
        for j in range(0, n_segments):
            # self loop
            self.add_edge(gmm_nodes[j], gmm_nodes[j], val=model.transitions[j, j])
            if j < n_segments - 1:
                # connect to the next
                self.add_edge(gmm_nodes[j], gmm_nodes[j + 1], val=model.transitions[j + 1, j])
        self.current = gmm_nodes[-1]

        # update node_index in all nodes
        self.update_node_indices()
        return self.current

    def update_node_indices(self):
        n_nodes = len(self.nodes)
        for i in range(n_nodes):
            if self.nodes[i].node_index is not None:
                assert self.nodes[i].node_index == i
            self.nodes[i].node_index = i

    def get_ends(self):
        return [self.current]


In [10]:
# continuous_train.py
from itertools import chain
import os
import pickle
import warnings
import copy


def continuous_train(data, models, lables, use_gmm=True, n_gaussians=4, use_em=True, max_iteration=1000):
    # remember old models
    old_models = copy.deepcopy(models)

    for iter in range(max_iteration):
        converged = True
        print('=' * 25)
        print('Continuous training iteration:', iter)
        print('Rearranging data for hmm training...')
        segments = {w: [] for w in chain.from_iterable(lables)}
        data_len = len(data)

        print('Building model graphs')
        # make model_graphs for current models
        model_graphs = [ContinuousGraph([]) for _ in lables]
        n_labels = len(lables)
        for i in range(n_labels):
            lb = lables[i]
            model_graphs[i].add_non_emitting_state()
            for digit in lb:
                model_graphs[i].add_model(old_models[digit], digit)
                model_graphs[i].add_non_emitting_state()

        for i in range(data_len):
            m = model_graphs[i]
            x = data[i]

            _, path = sentence_viterbi(x, m)
            print('Progress:', str(int(100 * i / data_len)) + "%", end='\r')

            # find segments for every digit, a segment is ended and started by 'NES'
            # each segment is a sequence of mfcc features, thus it's a 2d array-like
            i_nes = [i for i, x in enumerate(path) if x == "NES"]
            i_nes_len = len(i_nes)
            for index in range(i_nes_len - 1):
                start_i = i_nes[index] + 1
                segments[path[start_i]].append(x[start_i:i_nes[index + 1]])

        print('Complete data rearrangement')
        print("=" * 25)

        # remove templates that are too small to do dtw on
        for w in segments.keys():
            seg = segments[w]
            seg_len = len(seg)
            del_i = []
            for si in range(seg_len):
                if seg[si].shape[0] < 5:
                    warnings.warn('Removing digit templates that are too small', UserWarning)
                    del_i.append(si)

            del_i.sort(reverse=True)
            for si in del_i:
                del seg[si]

        # continuous training
        print('Doing HMM training...')
        for w in segments.keys():
            # train a new HMM model using the segments
            m = HMM(5)
            m.fit(segments[w], n_gaussians, use_gmm, use_em)
            converged = converged and m == old_models[w]
            old_models[w] = m
            # TODO: use command line argument for output model path
            file = open(os.path.join(continuous_model_path, str(w) + '.pkl'), 'wb')
            pickle.dump(m, file)
            file.close()

        if converged:
            print('Continuous training converged')
            break

In [11]:
# core.py
from scipy.io import wavfile
import numpy as np
import pickle
import os
import python_speech_features as psf
import re
import hashlib


def delta_feature(feat):
    delta = np.zeros(feat.shape)
    for i in range(len(feat)):
        if i == 0:
            delta[i] = feat[i + 1] - feat[i]
        elif i == len(feat) - 1:
            delta[i] = feat[i] - feat[i - 1]
        else:
            delta[i] = feat[i + 1] - feat[i - 1]
    return delta

def load_wav_as_mfcc(path):
    """
    Another version of load_wav_as_mfcc using library python_speech_feature to calculate
    mfcc, to check if this implementation is correct.
    """
    sample_rate, signal = wavfile.read(path)
    mfcc = psf.mfcc(signal, sample_rate, nfilt=40, preemph=0.95, appendEnergy=False, winfunc=np.hamming)
    df = delta_feature(mfcc)
    ddf = delta_feature(df)
    features = np.concatenate([mfcc, df, ddf], axis=1)
    features = standardize(features)
    return features


def make_HMM(filenames, n_segs, use_gmm, use_em):
    print('Loading wav files to mfcc features')
    ys = [load_wav_as_mfcc(filename) for filename in filenames]
    m = HMM(n_segs)
    print('Fitting HMMs')
    model = m.fit(ys, n_gaussians=4, use_gmm=use_gmm, use_em=use_em)
    return model


def train(filenames, model_folder, model_name, n_segs, use_gmm, use_em):
    models = make_HMM(filenames, n_segs, use_gmm=use_gmm, use_em=use_em)
    file = open(os.path.join(model_folder, model_name + '.pkl'), 'wb')
    pickle.dump(models, file)


def test(models, folder, file_patterns):
    n_passed = 0
    n_tests = 0

    # get the best model for every digit
    for digit in range(len(digit_names)):
        # get all test files using regular expresssions
        filenames = [os.path.join(folder, file) for file in os.listdir(folder) if
                     re.match(file_patterns[digit], file)]
        # update the total number of tests
        n_tests += len(filenames)

        # do evaluation on all models, find the best one
        # NOTE: the evaluation is based on costs
        for f in filenames:
            input = load_wav_as_mfcc(f)
            best_model = 0
            c = np.inf
            # find the best model
            for i in range(len(models)):
                m = models[i]
                cost = m.evaluate(input)
                if cost < c:
                    c = cost
                    best_model = i
            # if the best model is correct, move on
            if best_model == digit:
                n_passed += 1
            # if the best model is wrong, log info
            else:
                print("Digit:", digit_names[digit], "is wrong")
    return n_passed / n_tests


def aurora_continuous_train():
    models = []
    for digit in digit_names:
        # TODO: use command line argument for input model path
        file = open(os.path.join(isolated_model_path, str(digit) + '.pkl'), 'rb')
        models.append(pickle.load(file))
        file.close()

    # get filenames
    sequence_regex = re.compile('(?<=_)[OZ0-9]+(?=[AB])')
    filenames = [f for f in os.listdir(os.path.join(data_path, 'train')) if os.path.isfile(os.path.join(data_path, 'train', f))]
    filenames.sort()

    # get transcripts
    sequences = [re.search(sequence_regex, f).group(0) for f in filenames]
    labels = [list(map(lambda x: filename_index_map[x], s)) for s in sequences]


    print('loading data')
    data = [load_wav_as_mfcc(os.path.join(data_path, 'train', f)) for f in filenames]

    continuous_train(data, models, labels)

# Isolated Training

In [12]:
# print('Doing isolated training...')
# # data folder location
# folder = os.path.join(data_path, 'train')
# for digit in digit_names:
#     filenames = [os.path.join(folder, f) for f in os.listdir(folder) if
#                  re.match('[A-Z]+_' + digit + '[AB].wav', f)]
# 
#     train(filenames, isolated_model_path, digit, n_segs=5, use_gmm=True, use_em=True)

# Continuous Training

In [ ]:
aurora_continuous_train()